In [9]:
import numpy as np
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.layers.core import Flatten, Dense, Dropout
import pickle
from scipy import misc
from scipy.misc import imresize
from PIL import Image
import glob, os
import cv2
import csv
import math
import tensorflow as tf
from tqdm import tqdm
import matplotlib.pyplot as plt
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.layers import Convolution2D, MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
import json

In [2]:
learning_rate = 0.00001
batch_size = 20
nb_epoch = 3

In [3]:
model = Sequential()
model.add(Convolution2D(24, 5, 5,border_mode='valid',subsample=(2,2),  input_shape=(66,200,3)))
model.add(Convolution2D(36, 5, 5, border_mode='valid',subsample=(2,2)))
model.add(Convolution2D(48, 5, 5,subsample=(2,2)))
model.add(Convolution2D(64, 3, 3))
model.add(Convolution2D(64, 3, 3))
model.add(Flatten())
model.add(Dense((1164), activation='relu'))
model.add(Dense(100,activation='relu'))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1,name='output'))


In [4]:
images_list = []

for infile in glob.glob("IMG/*.jpg"):
    image = misc.imread(infile)
    image = misc.imresize(image, (66,200))
    images_list.append(image)
    
X_train = np.stack(images_list)
print(X_train.shape)

(2062, 66, 200, 3)


In [5]:
def normalize(image_data):
    """
    Normalize the image data with Min-Max scaling to a range of [-0.5, 0.5]
    :param image_data: The image data to be normalized
    :return: Normalized image data
    """
    #costants
    a = -0.5
    b = 0.5
    xmin = np.min(image_data)
    xmax = np.max(image_data) 
    
    x = image_data    
    x_prime = a + ((x-xmin)*(b-a))/(xmax-xmin)
    
    return x_prime

X_train = normalize(X_train)
print(X_train.shape)
print('Data normalized')

(2062, 66, 200, 3)
Data normalized


In [6]:
with open('driving_log.csv','r') as file:
    datareader = csv.reader(file,delimiter=',')
    driving_log = []
    for row in datareader:
        driving_log.append(row)
        
log = np.stack(driving_log)
#print(log.shape)

#labels
y_train = np.zeros(log.shape[0])
for i,row in enumerate(log):
    y_train[i] = log[i][3]
    
    
print (y_train.shape)

(2062,)


In [10]:
optimizer = Adam(lr=learning_rate)
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=nb_epoch, validation_split=0.3, shuffle=True)

Train on 1443 samples, validate on 619 samples
Epoch 1/3
1443/1443 [==============================] - 74s - loss: 0.0419 - acc: 0.7547 - val_loss: 0.0202 - val_acc: 0.7884
Epoch 2/3
1443/1443 [==============================] - 36s - loss: 0.0414 - acc: 0.7547 - val_loss: 0.0203 - val_acc: 0.7884
Epoch 3/3
1443/1443 [==============================] - 68s - loss: 0.0408 - acc: 0.7547 - val_loss: 0.0205 - val_acc: 0.7884


In [12]:
print(history.history['acc'][-1])

0.754677756495


In [13]:
json_string = model.to_json()
with open('model.json','w') as f:
    json.dump(json_string,f,ensure_ascii=False)

model.save_weights('model.h5')